In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Generic variables

numOfCoefficients = 286
numOfFeatures = 285
numOfSamples = 1000000
percentageOfValidation = 0.20
learningRate = 0.001
threshold = 0.5

# Define coefficient array and initialize with randoms

coefficients = np.random.randn(numOfCoefficients)

# Derived variables

numOfValiSamples = int(numOfSamples * percentageOfValidation)
numOfTrainSamples = numOfSamples - numOfValiSamples

# Load train data

train = np.loadtxt(fname='train.csv/train.csv', delimiter=',', max_rows=(numOfSamples), skiprows=1)

# Data arrays

trainAllFeatures = train[numOfValiSamples:, 1:-1]
trainTargets = train[numOfValiSamples:, -1]
valiAllFeatures = train[:numOfValiSamples, 1:-1]
valiTargets = train[:numOfValiSamples, -1]
accuracy = []

In [ ]:
# Sigmoid Function

def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [ ]:
# This function can calculate z from given coefficients and features

def calculateZ(coefficients, features):
    sum = coefficients[0]
    for i in range (numOfFeatures):
        sum += features[i] * coefficients[i + 1]
    return sum

# This function return f cap

def calculateYCap(coefficients, features):
    return sigmoid(calculateZ(coefficients, features))

In [ ]:
def logLikelihoodSub(coefficients, features, target):
    yCap = calculateYCap(coefficients, features)
    epsilon = 1e-7
    yCap = np.maximum(np.full(yCap.shape, epsilon), np.minimum(np.full(yCap.shape, 1-epsilon), yCap))
    return np.sum(target*np.log(yCap)+(1-target)*np.log(1-yCap))

def logLikelihood(coefficients, allFeatures, targets):
    llhood = 0
    for i in range(numOfCoefficients):
        llhood *= logLikelihoodSub(coefficients, allFeatures[i], targets[i])
    return llhood

In [ ]:
# Accuracy Calculator
def calculateAccuracyOneSample(coefficients, features, target):
    YCap = calculateYCap(coefficients, features)
    isItHit = False
    
    if YCap >= threshold:
        prediction = 1
    else:
        prediction = 0
    
    if prediction == target:
        isItHit = True
    
    return 1 if isItHit else 0

def calculateAccuracy(coefficients, valiAllFeatures, valiTargets):
    hit = 0
    
    for i in range(numOfValiSamples):
        hit += calculateAccuracyOneSample(coefficients, valiAllFeatures[i], valiTargets[i])
    
    return hit/numOfValiSamples


In [ ]:
# Stochastic Gradient Ascent Implementation

def SGAOneFeature(residual, Bj, Xj, learningrate):
    return Bj + learningrate * (residual * Xj)
def SGAOneFeatureB0(residual, B0, learningrate):
    return B0 + learningrate * residual

def SGAOneSample(coefficients, features, target, learningrate):
    residual = target - calculateYCap(coefficients, features)

    updatedCoefficients = coefficients
    updatedCoefficients[0] = SGAOneFeatureB0(residual, coefficients[0], learningrate)
    for i in range(1, numOfCoefficients):
        updatedCoefficients[i] = SGAOneFeature(residual, coefficients[i], features[i-1], learningrate)
    
    return updatedCoefficients
        
def SGAOneEpoch(coefficients, allFeatures, targets, learningrate):
    
    for i in range(numOfTrainSamples):
        #print(i)#debug
        coefficients = SGAOneSample(coefficients, allFeatures[i], targets[i], learningrate)
    
    return coefficients

def SGA(coefficients, allFeatures, targets, learningrate):
    
    print ("Initial Accuracy : ", calculateAccuracy(coefficients, valiAllFeatures, valiTargets))
    for i in range (1000):
        coefficients = SGAOneEpoch(coefficients, allFeatures, targets, learningrate)
        accuracy.append(calculateAccuracy(coefficients, valiAllFeatures, valiTargets))
        print ("Epoch ", (i + 1) ,": Accuracy : ", accuracy[-1])

SGA(coefficients, trainAllFeatures, trainTargets, learningRate)

In [ ]:
plt.plot(range(len(accuracy)), accuracy)
plt.xlabel("Batch")
plt.ylabel("Accuracy")
plt.title("BATCH GRADIENT ASCENT")
plt.show()

In [ ]:
# Minibatch Gradient Ascent Implementation (Can be used for batch too)

batchAmount = 10
batchSize = int(numOfTrainSamples/batchAmount)
        
def MGAOneBatch(coefficients, allFeatures, targets, learningrate, batchSize):
    
    #Initialize residuals
    residuals = np.zeros(batchSize)
    for i in range(batchSize):
        residuals[i] = targets[i] - calculateYCap(coefficients, allFeatures[i])
    
    coefficients[0] += learningrate * np.sum(residuals)
    
    for j in range(numOfCoefficients - 1):
        coefficients[j + 1] += learningrate * np.dot(residuals, allFeatures[:, j])
        
    return coefficients

def MGAOneEpoch(coefficients, allFeatures, targets, learningrate, batchSize):
    
    for i in range(int(numOfTrainSamples/batchSize)):
        coefficients = MGAOneBatch(coefficients, allFeatures[batchSize*i:batchSize*(i+1)], targets[batchSize*i:batchSize*(i+1)], learningrate, batchSize)
    return coefficients
    
    
def MGA(coefficients, allFeatures, targets, learningrate, batchSize):
    
    print ("Initial Accuracy : ", calculateAccuracy(coefficients, valiAllFeatures, valiTargets))
    for i in range (1000):
        coefficients = MGAOneEpoch(coefficients, allFeatures, targets, learningrate, batchSize)
        accuracy.append(calculateAccuracy(coefficients, valiAllFeatures, valiTargets))
        print ("Epoch ", (i + 1) ,": Accuracy : ", accuracy[-1])

MGA(coefficients, trainAllFeatures, trainTargets, learningRate, batchAmount)

In [ ]:
plt.plot(range(len(accuracy)), accuracy)
plt.xlabel("Batch")
plt.ylabel("Accuracy")
plt.title("Batch size 10, Learning rate 0.001 Gradient Ascent")
plt.show()